In [1]:
#Update code

In [11]:
import sys
print(sys.executable)

E:\Python\envs\tf_2_10\python.exe


In [13]:
#CodeCarbon Test
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tqdm import tqdm

In [14]:
# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize pixel values to [0, 1]
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convert labels to one-hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [16]:
# Load pre-trained model
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(32, 32, 3),
    include_top=False,
    weights='imagenet'
)

# Fine-tune the model
model = Sequential([
    tf.keras.layers.Resizing(32, 32, input_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(3, (3, 3), padding='same'),
    base_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

model.fit(
    x_train, y_train,
    epochs=5,
    validation_data=(x_test, y_test))

9406464/9406464 [==============================] - 7s 1us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 32, 32, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 3)         30        
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 1, 1, 1280)       2257984   
 ional)                                                          
                                                                 
 flatten (Flatten)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 128)               163968    
                                                                 
 dense_1 (Dense)             (None, 10)                1290

In [17]:
# Get teacher model predictions
teacher_predictions = model.predict(x_train)

# Create TensorFlow Dataset for training data and teacher predictions
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)
teacher_predictions_dataset = tf.data.Dataset.from_tensor_slices(
    teacher_predictions).batch(32)

# Combine the two datasets into one
train_dataset_with_teacher_preds = tf.data.Dataset.zip(
    (train_dataset, teacher_predictions_dataset))

# Define the student model
student_model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

student_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

student_model.summary()

1875/1875 [==============================] - 15s 8ms/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               100480    
                                                                 
 dense_3 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [18]:
from codecarbon import EmissionsTracker

# Measure emissions for transfer learning model inference
tracker = EmissionsTracker()
tracker.start()
model.evaluate(x_test, y_test)
emissions_transfer_learning = tracker.stop()

# Measure emissions for knowledge distillation model inference
tracker = EmissionsTracker()
tracker.start()
student_model.evaluate(x_test, y_test)
emissions_knowledge_distillation = tracker.stop()

print(f"Emissions for transfer learning: {emissions_transfer_learning} kg CO2")
print(f"Emissions for knowledge distillation: {emissions_knowledge_distillation} kg CO2")

[codecarbon WARNING @ 10:42:39] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 10:42:39] [setup] RAM Tracking...
[codecarbon INFO @ 10:42:39] [setup] CPU Tracking...
[codecarbon WARNING @ 10:42:41] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Windows OS detected: Please install Intel Power Gadget to measure CPU

[codecarbon INFO @ 10:42:41] CPU Model on constant consumption mode: AMD Ryzen 9 5900XT 16-Core Processor
[codecarbon WARNING @ 10:42:41] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 10:42:41] [setup] GPU Tracking...
[codecarbon INFO @ 10:42:41] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 10:42:41] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: pynvml
            
[codecarbon INFO @ 10:42:41] >>> Tracker's metad

313/313 [==============================] - 3s 11ms/step - loss: 1.8566 - accuracy: 0.9008


[codecarbon INFO @ 10:42:49] Energy consumed for RAM : 0.000019 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 10:42:49] Delta energy consumed for CPU with constant : 0.001602 kWh, power : 1680.0 W
[codecarbon INFO @ 10:42:49] Energy consumed for All CPU : 0.001602 kWh
[codecarbon INFO @ 10:42:49] Energy consumed for all GPUs : 0.000015 kWh. Total GPU Power : 15.7559409623872 W
[codecarbon INFO @ 10:42:49] 0.001636 kWh of electricity used since the beginning.
[codecarbon WARNING @ 10:42:49] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 10:42:49] [setup] RAM Tracking...
[codecarbon INFO @ 10:42:49] [setup] CPU Tracking...
[codecarbon WARNING @ 10:42:50] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Windows OS detected: Please install Intel Power Gadget to measure CPU

[codecarbon INFO @ 10:42:50] CPU Model on constant consumption mode: AMD Ryzen 9 5900XT 16-Core Processor
[codecarbon WARNING @ 10:42:50] No CPU track

313/313 [==============================] - 1s 3ms/step - loss: 2.4318 - accuracy: 0.0713


[codecarbon INFO @ 10:42:56] Energy consumed for RAM : 0.000007 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 10:42:56] Delta energy consumed for CPU with constant : 0.000560 kWh, power : 1680.0 W
[codecarbon INFO @ 10:42:56] Energy consumed for All CPU : 0.000560 kWh
[codecarbon INFO @ 10:42:56] Energy consumed for all GPUs : 0.000004 kWh. Total GPU Power : 10.53593594156005 W
[codecarbon INFO @ 10:42:56] 0.000570 kWh of electricity used since the beginning.


Emissions for transfer learning: 3.890286553768168e-06 kg CO2
Emissions for knowledge distillation: 1.3544507325187316e-06 kg CO2
